In [1]:
#Module imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import torch.nn.functional as F

/data/rsg/chemistry/johnyang/miniconda3/envs/ls/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200

In [3]:
toy_dataset = pd.read_csv('../data/2_class.csv')
classes = pd.unique(toy_dataset['family_accession'])

In [4]:
len(classes) #Should be 2

2

In [5]:
# Irrelevant. Example. We will use ESMFnDataset defined below.
class SeqFnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes):
        self.dataset = dataset
        self.classes = classes
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.idx_to_class = {i: classes[i] for i in range(len(classes))}
        self.letter_to_num = {'C': 4, 'D': 3, 'S': 15, 'Q': 5, 'K': 11, 'I': 9,
                'P': 14, 'T': 16, 'F': 13, 'A': 0, 'G': 7, 'H': 8,
                'E': 6, 'L': 10, 'R': 1, 'W': 17, 'V': 19, 
                'N': 2, 'Y': 18, 'M': 12}
        self.num_to_letter = {v:k for k, v in self.letter_to_num.items()}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        seq = torch.as_tensor([self.letter_to_num[a] for a in row['sequence']],
                    device='cpu', dtype=torch.long)
        class_idx = torch.tensor(self.class_to_idx[row['family_accession']])
        label = F.one_hot(class_idx, num_classes=len(self.classes))
        return seq, label


In [6]:
dl = torch.utils.data.DataLoader(SeqFnDataset(toy_dataset, classes))

In [7]:
for batch in dl:
    print(batch)
    break

[tensor([[19, 10,  3,  9,  0,  7,  7, 16,  7,  3, 19, 16, 13, 16, 19,  0,  1, 16,
         11,  8, 14,  1,  8,  9,  5,  4, 16,  3, 10, 19, 14,  6, 12, 10,  3, 19,
          0,  1, 12,  8,  9,  3,  6,  7,  3,  7,  0,  7, 19, 14,  9,  3, 13,  6,
         19, 19,  3,  0,  5,  2,  9, 14, 18,  6,  3,  0, 15, 18,  3, 19,  9, 16,
         12,  0, 18,  7,  9,  1,  2, 12, 14,  3,  1, 14,  1,  0, 10, 15,  6, 12,
         13,  1, 19, 10, 11, 14,  7,  7]]), tensor([[1, 0]])]


In [15]:
import esmdir


In [16]:
from esmdir.esm.model.esm2 import ESM2

AttributeError: module 'esm' has no attribute 'data'

In [ ]:
model_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D.pt')
regression_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D-contact-regression.pt')

In [10]:
from utils.ls_utils import *

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from utils.gpu_utils import *

In [ ]:
chosen_gpu = get_free_gpu()
device = 'cuda'

cuda
Using GPUs: 0


In [ ]:
esm_model, alphabet = load_model_and_alphabet_core("esm2_t33_650M_UR50D", model_data, regression_data)
batch_converter = alphabet.get_batch_converter()
esm_model.to(device)
esm_model.eval()
print('done')

done


In [ ]:
for row in toy_dataset.iterrows():
    seq = row[1]['sequence']
    print(row)
    print(seq)
    break

(0, Unnamed: 0                                                          0
index                                                             910
family_id                                             Methyltransf_25
sequence_name                                     S6CDL6_9ACTN/62-159
family_accession                                            PF13649.6
aligned_sequence    VLDIAGGT.G...D..VT...FT..VA.RT...........KH......
sequence            VLDIAGGTGDVTFTVARTKHPRHIQCTDLVPEMLDVARMHIDEGDG...
Name: 0, dtype: object)
VLDIAGGTGDVTFTVARTKHPRHIQCTDLVPEMLDVARMHIDEGDGAGVPIDFEVVDAQNIPYEDASYDVITMAYGIRNMPDRPRALSEMFRVLKPGG


In [ ]:
def pad_seq_to_max_len(seq, max_len):
    padding_token = '<pad>'
    seq = seq + padding_token * (max_len - len(seq))
    return seq

We want to pad sequences to all be <= a defined length.

In [ ]:
data = [(row[1]['sequence_name'], pad_seq_to_max_len(row[1].sequence, protein_len)) for row in toy_dataset.iterrows()]

In [ ]:
len(data)

6952

In [ ]:
import tqdm

KEY AREA TO IMPROVE:

Here we save the embeddings in a directory ran one-by-one... This is extremely inefficient given we have 1 million sequences... \

The below script will extract embeddings given a Fasta file efficiently.

python esm/scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
    examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

You can write Fasta files from pandas dataframes easily. Here's an example: https://bootstrappers.umassmed.edu/guides/main/r_writeFasta.html

In [ ]:
'''
RAN ALREADY FOR TOY EXAMPLE

for i, protein in tqdm.tqdm(enumerate(data)):
    batch_labels, batch_strs, batch_tokens = batch_converter([protein])
    batch_tokens = batch_tokens.to(device)
    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
    embeddings = results['representations'][33].detach().cpu().numpy()
    sequence_name = protein[0]
    # Modify all slashes in sequence name to dashes or sees name as directory, perhaps this is a good thing...
    sequence_name = sequence_name.replace('/', '-')
    torch.save(embeddings, f'../data/toy_esm_embeddings/{sequence_name}.pt')
'''

"\nRAN ALREADY FOR TOY EXAMPLE\n\nfor i, protein in tqdm.tqdm(enumerate(data)):\n    batch_labels, batch_strs, batch_tokens = batch_converter([protein])\n    batch_tokens = batch_tokens.to(device)\n    with torch.no_grad():\n        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)\n    embeddings = results['representations'][33].detach().cpu().numpy()\n    sequence_name = protein[0]\n    # Modify all slashes in sequence name to dashes or sees name as directory, perhaps this is a good thing...\n    sequence_name = sequence_name.replace('/', '-')\n    torch.save(embeddings, f'../data/toy_esm_embeddings/{sequence_name}.pt')\n"

In [ ]:
sequence_name = row[1]['sequence_name']
sequence_name = sequence_name.replace('/', '-')

In [ ]:
sequence_name

'S6CDL6_9ACTN-62-159'

In [ ]:
ex = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')

In [ ]:
ex.shape

(1, 202, 1280)

In [ ]:
class ESMFnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, device='cuda', max_len=200):
        self.dataset = dataset
        self.classes = classes
        self.max_len = max_len
        self.device = device
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.idx_to_class = {i: classes[i] for i in range(len(classes))}
        self.letter_to_num = {'C': 4, 'D': 3, 'S': 15, 'Q': 5, 'K': 11, 'I': 9,
                'P': 14, 'T': 16, 'F': 13, 'A': 0, 'G': 7, 'H': 8,
                'E': 6, 'L': 10, 'R': 1, 'W': 17, 'V': 19, 
                'N': 2, 'Y': 18, 'M': 12}
        self.num_to_letter = {v:k for k, v in self.letter_to_num.items()}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        sequence_name = row['sequence_name']
        sequence_name = sequence_name.replace('/', '-')
        embeddings = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')
        embeddings = torch.tensor(embeddings, device='cuda', dtype=torch.float32)

        '''Pad embeddings to max_len with zero vector'''
        if embeddings.size(1) < self.max_len:
            B, N, h = embeddings.size()
            pad = torch.zeros((B, self.max_len - embeddings.shape[1], h), device=self.device)
            embeddings = torch.cat((embeddings, pad), dim=1)

        class_idx = torch.tensor(self.class_to_idx[row['family_accession']])
        label = F.one_hot(class_idx, num_classes=len(self.classes))
        return embeddings, label

In [ ]:
dataset = ESMFnDataset(toy_dataset, classes, device)

In [ ]:
esmdl = torch.utils.data.DataLoader(dataset)

In [ ]:
for batch in esmdl:
    print(batch[0][0])
    break

tensor([[[ 0.0243,  0.0271, -0.0837,  ..., -0.2799,  0.1995,  0.0577],
         [-0.0919, -0.0847, -0.0205,  ..., -0.0418,  0.1840, -0.1400],
         [ 0.1741,  0.1002,  0.0078,  ...,  0.0821,  0.0727, -0.0905],
         ...,
         [-0.0306, -0.0665,  0.0843,  ..., -0.0120, -0.0695, -0.1188],
         [-0.0623,  0.0503,  0.0476,  ...,  0.0348, -0.0654, -0.0851],
         [-0.0286,  0.0321,  0.0917,  ..., -0.0120, -0.0180, -0.0668]]],
       device='cuda:0')


In [ ]:
class TransformerEncoder(torch.nn.Module):
    
    def __init__(self, input_size=1280, nheads=8, num_layers=6, device='cuda', classes=2, max_len=202, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.input_size = input_size
        self.classes = classes
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.input_size, nhead=nheads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc1 = torch.nn.Linear(self.input_size, 256)
        self.fc2 = torch.nn.Linear(256, 32)
        self.fc3 = torch.nn.Linear(32 * max_len, self.classes)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.device = device
        
    def forward(self, embedding):
        if len(embedding.size()) > 3 and embedding.size(0) == 1:
            embedding = embedding.squeeze(0)
            assert len(embedding.size()) == 3, 'Embedding has greater than 4 dimensions'
            
        B, N, h = embedding.shape
        hidden = self.transformer_encoder(embedding)
        hidden = self.fc1(hidden)
        hidden = self.fc2(hidden)
        '''Flatten hidden state'''
        hidden = hidden.view(B, -1)
        hidden = self.fc3(hidden)
        hidden = self.softmax(hidden)
        return hidden

In [ ]:
model = TransformerEncoder(input_size=1280, nheads=8, num_layers=6, device=device, classes=len(classes)).to(device)

In [ ]:
len(classes)

2

In [ ]:
for batch in esmdl:
    embedding, label = batch
    pred = model(embedding)
    print(pred.shape) # shape is [batch_size, number of classes]
    break

torch.Size([1, 2])


# Learning to Split

In [ ]:
import ls

: 

In [ ]:
# python scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
#   examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

: 